In [ ]:
import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange import Interchange
from openff.interchange.models import PotentialKey, TopologyKey
from openff.interchange.tests import get_test_file_path

In [ ]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField("openff-1.0.0.offxml")

top = Topology.from_molecules(2 * [Molecule.from_smiles("CCO")])

In [ ]:
# Use a monkey-patched function to parametrize the topology against a force field
sys_out = Interchange.from_smirnoff(force_field=parsley, topology=top)

In [ ]:
# Look at which ParameterHandler objects from the OpenFF toolkit
# have been made into Potentialhandler objects
sys_out.handlers.keys()

In [ ]:
# Store the bond handler to inspect its contents
bonds = sys_out.handlers["Bonds"]

In [ ]:
# Look at some ~metadata
bonds.type, bonds.expression, bonds.independent_variables

In [ ]:
# Return a mapping between atom indices and SMIRKS identifiers
bonds.slot_map

In [ ]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# bonds in the topology and unique parameters in the force field
bonds.potentials

In [ ]:
# Look at this contents of one of the Potential objects
potential_key = PotentialKey(id="[#6X4:1]-[#1:2]", associated_handler="Bonds")
bonds.potentials[potential_key]

In [ ]:
# Further, look at the particular value of one of its parameters
bonds.potentials[potential_key].parameters["k"]

In [ ]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
topology_key = TopologyKey(atom_indices=(1, 7))
sys_out.handlers["Bonds"].potentials[
    sys_out.handlers["Bonds"].slot_map[topology_key]
].parameters["k"]